# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [2]:
# your code here
import pandas as pd
data = pd.read_csv('tic-tac-toe.csv')

In [3]:
data.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


In [8]:
for column in data.columns:
    data[column] = data[column].replace('b',0).replace('x',1).replace('o',2)

In [10]:
y = data['class']
X = data.drop('class',axis=1)

In [11]:
X_norm = X/2

In [30]:
X_norm.shape

(958, 9)

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [49]:
# your code here
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, random_state=0)

In [50]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()

In [51]:
model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])

In [52]:
model.fit(X_train,y_train)

24/24 [==============================] - 0s 1ms/step - loss: 5.8859 - accuracy: 0.3290


In [53]:
model.evaluate(X_test,y_test)

6/6 [==============================] - 0s 2ms/step - loss: 5.4945 - accuracy: 0.3438


[5.494471073150635, 0.34375]

In [54]:
model.save_weights('tic-tac-toe.model')

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [55]:
# your code here
model.load_weights('tic-tac-toe.model')

In [67]:
predictions = model.predict(y_test[:5])

In [68]:
import numpy as np
print(predictions)
print(y_test[:5])

[[False]
 [ True]
 [ True]
 [ True]
 [ True]]
879    False
496     True
14      True
546     True
55      True
Name: class, dtype: bool


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [227]:
import tensorflow as tf

In [228]:
# your code here
model = Sequential()
model.add(tf.keras.layers.Dense(2))
tf.keras.optimizers.Adam(learning_rate=0.001)

In [229]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [230]:
model.fit(X_train,y_train,epochs=40)

Epoch 1/40

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

24/24 [==============================] - 0s 1ms/step - loss: 0.7182 - accuracy: 0.5104
Epoch 2/40
24/24 [==============================] - 0s 2ms/step - loss: 0.6892 - accuracy: 0.5131
Epoch 3/40
24/24 [==============================] - 0s 3ms/step - loss: 0.6809 - accuracy: 0.5313
Epoch 4/40
24/24 [==============================] - 0s 2ms/step - loss: 0.6591 - accuracy: 0.5444
Epoch 5/40
24/24 [==============================] - 0s 1ms/step - loss: 0.6543 - accuracy: 0.5561
Epoch 6/40
24/24 [

In [231]:
model.evaluate(X_test,y_test)

6/6 [==============================] - 0s 1ms/step - loss: 0.6233 - accuracy: 0.6146


[0.6233339309692383, 0.6145833134651184]

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here
#I believe the epochs helped more to improve the model.